In [10]:
%load_ext autoreload
%autoreload 1

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_encoder

from PIL import Image
import os
from pathlib import Path
import glob
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm, trange

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as T

%aimport modules
from modules.DataScienceBowl import DataScienceBowl
from modules.Trainer import Trainer
from modules.CustomUnet import CustomUnet
from modules.Transforms import CombiTransform, ElasticTransform
%aimport modules.denseCL.DenseCL
from modules.denseCL.DenseCL import DenseCL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Set device to GPU if available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('using cuda')
else:
    device = torch.device('cpu')
    print('using cpu')
    

# Set seeds
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

using cuda


In [ ]:
# class DenseCL(torch.nn.Module):
    
#     def __init__(self, lam=0.5, combination=None):
#         super().__init__()
        
#         self.Lambda = lam
        
#         if combination is not None:
#             self.transformer = CombiTransform(combination)
#         else:
#             self.transformer = CombiTransform()
            
#         self.encoder = get_encoder('resnet18', 3, 5, None)
# #         self.encoder = get_encoder('resnet18', 3, 5, 'imagenet')
#         self.encoder.layer4 = torch.nn.Identity()
        
#         self.project = DenseContrastiveModule()
        
# #         self.enc_proj = DenseContrastiveModule()
        
#         self.dense_loss = DenseContrastiveLoss()
#         self.glob_loss = GlobalContrastiveLoss()
        
        
#     def forward(self, image_1, image_2):
        
#         img_Q = self.transformer(image_1)
#         img_pos = self.transformer(image_1)
#         img_neg = self.transformer(image_2)
        
#         img_Q = self.encoder(img_Q)[-1]
#         img_pos = self.encoder(img_pos)[-1]
#         img_neg = self.encoder(img_neg)[-1]
        
#         dense_Q, glob_Q = self.project(img_Q)
#         dense_pos, glob_pos = self.project(img_pos)
#         dense_neg, glob_neg = self.project(img_neg)
        
#         d_loss = self.dense_loss(dense_Q, dense_pos, dense_neg)
#         g_loss = self.glob_loss(glob_Q, glob_pos, glob_neg)
        
#         loss = (1 - self.Lambda) * g_loss + self.Lambda * d_loss
        
#         return loss
        
        


# class DenseContrastiveModule(torch.nn.Module):
    
#     def __init__(self):
#         super().__init__()
#         # Reduce bottleneck from 14x14x256 to 5x5x128
#         self.densePool = torch.nn.MaxPool2d(2)
#         self.projDense = torch.nn.Conv2d(256, 128, (3,3))
        
#         self.projGlobal = torch.nn.AvgPool2d(14)
#         self.batchNorm = torch.nn.BatchNorm2d(256)
#         pass
    
#     def forward(self, input):
# #         encoded = self.encoder(input)[-1]
#         normalized = self.batchNorm(input)
        
#         pooled = self.densePool(normalized)
#         dense = self.projDense(pooled)
#         glob = self.projGlobal(normalized)

# #         loss = GlobalContrastiveLoss()
# #         loss(glob, glob, glob)
        
#         return dense, glob

    
# #  Temperature
# #  Lambda -> more to global
# #  Bottleneck resolution (3,3 or 5,5)
# #  Weight decay (optimizer) [0.0001] both contra and target task
# #  Save for each 5 epochs the weights
# #  Smaller learning rate (0.0001)


In [ ]:


# class DenseContrastiveLoss(torch.nn.Module):

#     def __init__(self):
#         super().__init__()

#     def forward(self, dense_img, dense_pos, dense_neg):
        
#         def reshape_bottleneck(x):
#             return x.reshape(x.shape[0], x.shape[1], -1)
    

#         dense_img = reshape_bottleneck(dense_img)
#         dense_pos = reshape_bottleneck(dense_pos)
#         dense_neg = reshape_bottleneck(dense_neg)
        
#         assert dense_img.shape == dense_pos.shape and dense_img.shape == dense_neg.shape, "input shapes should be the same"
        
#         B = dense_img.shape[0]
#         D = dense_img.shape[1]
#         S = dense_img.shape[2]
        
#         temperature = 50
        
#         loss_sum = 0
#         for i in range(S):
#             query = dense_img[:,:,i]

#             # Find most similar target vector
#             dist = F.cosine_similarity(dense_pos, query.unsqueeze(-1), dim=1)
#             max_i = torch.argmax(dist, dim=1)
#             batch_pos = torch.cat([ torch.index_select(a, 1, i).unsqueeze(0) for a, i in zip(dense_pos, max_i) ])
            
#             dot_pos = torch.bmm(query.view(B,1,D), batch_pos.view(B,D,1))
#             exp_pos = torch.exp(dot_pos / temperature)
            
#             dot_neg = torch.bmm(query.view(B,1,D), dense_neg.view(B,D,S))
#             exp_neg = torch.exp(dot_neg/temperature)
#             sum_neg = torch.sum(exp_neg, axis=2, keepdim=True)
            
#             loss_sum += -torch.log(exp_pos / (exp_pos + sum_neg))


#         return torch.mean(loss_sum / S)
            
        
# class GlobalContrastiveLoss(torch.nn.Module):

#     def __init__(self):
#         super().__init__()

#     def forward(self, global_img, global_pos, global_neg):
    
#         assert global_img.shape == global_pos.shape and global_img.shape == global_neg.shape, "input shapes should be the same"
                
#         B = global_img.shape[0]
#         D = global_img.shape[1]
        
#         temperature = 50
        
# #         for i in range(S):
#         query = global_img

#         # Find most similar target vector
#         dist = F.cosine_similarity(global_pos, query, dim=1)
#         max_i = torch.argmax(dist, dim=1)
#         batch_pos = torch.cat([ torch.index_select(a, 1, i).unsqueeze(0) for a, i in zip(global_pos, max_i) ])

#         dot_pos = torch.bmm(query.view(B,1,D), batch_pos.view(B,D,1))
#         exp_pos = torch.exp(dot_pos / temperature)

#         dot_neg = torch.bmm(query.view(B,1,D), global_neg.view(B,D,1))
#         exp_neg = torch.exp(dot_neg/temperature)
#         sum_neg = torch.sum(exp_neg, axis=2, keepdim=True)

#         global_loss = -torch.log(exp_pos / (exp_pos + sum_neg))

#         return torch.mean(global_loss)

    
    
# # - Weight decay in optimizer



In [3]:
# Define Transforms
# TODO Separate Train and Test transforms
transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
#     T.ToTensor()
#     T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

In [12]:
# Load data and split into train and test

seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

dataset = DataScienceBowl('data/data_science_train', transform=transform)

indices = torch.randperm(len(dataset)).tolist()
N = len(indices)
# [ 0.3 train | 0.1 test | 0.1 validate | 0.5 unlab ]
# train_test, test_val, val_unlab = map(int, (0.3 * N, 0.4 * N, 0.5 * N))

# [ 0.2 train | 0.05 test | 0.05 validate | 0.5 unlab ]
train_test, test_val, val_unlab = map(int, (0.2 * N, 0.25 * N, 0.3 * N))

# train_test, test_val, val_unlab = (8, 16, 24)


train_dataset = torch.utils.data.Subset(dataset, indices[:train_test])
test_dataset = torch.utils.data.Subset(dataset, indices[train_test:test_val])
val_dataset = torch.utils.data.Subset(dataset, indices[test_val:val_unlab])
unlabeled_dataset = torch.utils.data.Subset(dataset, indices[val_unlab:])

print(len(unlabeled_dataset))

BATCH_SIZE = 8
dataLoader_training = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataLoader_test = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataLoader_unlabeled = DataLoader(dataset=unlabeled_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataLoader_validation = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)

469


In [ ]:

import pylab as pl
from IPython import display

# model = DenseCL(combination=['color', 'random', 'noise'])


In [13]:

pretrain_hyper_params = {
    'lr': [0.01, 0.001, 0.0001],
    'lambda': [0, 0.5, 1],
    'dense_head': ['5x5', '3x3']
}


for lr in pretrain_hyper_params['lr']:
    for lam in pretrain_hyper_params['lambda']:
        for dense_head in pretrain_hyper_params['dense_head']:
            
            model = DenseCL(lam=lam, dense_head=dense_head)
            model.to(device)


            # Use Binary Cross Entropy Loss and Stochastic Gradient Descent
            # criterion = torch.nn.BCEWithLogitsLoss()
            optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=0.0001)


            epochs = 50
            # epoch = 0
            progressbar = trange(epochs, desc="Progress")

            training_loss = list()

            for epoch in progressbar:

                model.train()

                epoch_losses = list()
                batch_iter = tqdm(enumerate(dataLoader_unlabeled), "Self Supervised Training",
                                 total=len(dataLoader_unlabeled), leave=False)

                for i, sample in batch_iter:
                    x, y = sample.values()

                    batch, target = x.to(device), y.to(device)

                    B = batch.shape[0]

                    if B % 2 != 0:
                        continue


                    image_1, image_2 = torch.split(batch, B//2)

                    optimizer.zero_grad()
                    loss = model(image_1, image_2)

                    epoch_losses.append(loss.item())
                    loss.backward()
                    optimizer.step()

                    batch_iter.set_description(f'Training: (loss {loss.item():.4f})')


                training_loss.append(np.mean(epoch_losses))
#                 print(training_loss[-1])   
                batch_iter.close()

                # Store model every 5 epochs for later use
                if (epoch + 1) % 5 == 0:
                    print(f"storing model at {epoch+1}")
                    model_name = f"models/DenseCL_GRID_{dense_head}_E-{epoch+1}_LR-{lr}_LAM-{lam}.pt"
                    torch.save(model.state_dict(), model_name)
                    
            break
    
    
    

['elastic', 'color', 'random', 'noise']


Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Self Supervised Training:   0%|          | 0/59 [00:00<?, ?it/s]

Self Supervised Training:   0%|          | 0/59 [00:00<?, ?it/s]

Self Supervised Training:   0%|          | 0/59 [00:00<?, ?it/s]

...

storing model at 30


Self Supervised Training:   0%|          | 0/59 [00:00<?, ?it/s]

Self Supervised Training:   0%|          | 0/59 [00:00<?, ?it/s]

Self Supervised Training:   0%|          | 0/59 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'DenseCL_rand_5x5dense_50epochs_lr0_001.pt')

In [ ]:
plt.plot(training_loss)
# plt.plot(val_loss)
# plt.hlines(test_loss, 0, len(train_loss) - 1, color='red')
plt.show()

In [ ]:
unet = CustomUnet(model.encoder).to(device)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(unet.parameters(), lr=0.001)

trainer = Trainer(model=unet,
                 device=device,
                 criterion=criterion,
                 optimizer=optimizer,
                 training_DataLoader=dataLoader_training,
                 validation_DataLoader=dataLoader_validation,
                 test_DataLoader=dataLoader_test,
                 epochs=20,
                 notebook=True,
                 seed=42)

train_loss, val_loss, lr, test_loss, test_iou = trainer.run_trainer()

In [ ]:
print('Test IoU', np.round(test_iou, 4))
plt.plot(train_loss)
plt.plot(val_loss)
plt.hlines(test_loss, 0, len(train_loss) - 1, color='red')
plt.show()

In [ ]:
# unet = CustomUnet(model.encoder)

i = int(random.random() * len(test_dataset))
# i = 55
sample = train_dataset[i]
img, target = sample.values()
unet.eval()
out = unet(img.unsqueeze(0).to(device))

print(torch.max(out))
print(torch.min(out))

f,a = plt.subplots(1,2)

a[0].imshow(out.squeeze(0).permute(1,2,0).detach().cpu().numpy())
a[1].imshow(img.permute(1,2,0))
plt.show()


In [ ]:
import torch.nn.functional as F

mask = F.sigmoid(out) >= 0.5

mask = mask.squeeze(0).squeeze(0).detach().cpu().numpy()

target = target.squeeze(0).type(torch.BoolTensor).detach().numpy()

f, a = plt.subplots(1,2)

a[0].imshow(mask)
a[1].imshow(target)

# plt.imshow(mask)
plt.show()

In [ ]:
# IoU
from sklearn.metrics import jaccard_score

jaccard_score(mask.reshape(-1), target.reshape(-1))

In [ ]:
f,a = plt.subplots(1,2, figsize=(15,10))

a[0].imshow((mask ^ target) & mask)
a[1].imshow((mask ^ target) & target)
